In [12]:
import pandas as pd
from gurobipy import Model, GRB, quicksum

# Load and clean your data
data = pd.read_csv('/Users/cindychang/Documents/school/大二/OR/midterm/data/instance05.csv')

def parse_machine_list(machine_list):
    if pd.isna(machine_list):
        return []
    return list(map(int, machine_list.split(',')))

data['Stage-1 Machines'] = data['Stage-1 Machines'].apply(parse_machine_list)
data['Stage-2 Machines'] = data['Stage-2 Machines'].apply(parse_machine_list)

# set up all the machine's id
all_machines = set()
data['Stage-1 Machines'].apply(lambda machines: all_machines.update(machines))
data['Stage-2 Machines'].apply(lambda machines: all_machines.update(machines))


# Initialize the model
model = Model("Job_Scheduling")

# Extract jobs, machines, and assume stages and data are set
jobs = data['Job ID'].unique()
machines = set(sum(data['Stage-1 Machines'].tolist(), []) + sum(data['Stage-2 Machines'].tolist(), []))

# Define a large M
M = 10000  # Adjust based on your expected schedule bounds

# Add decision variables
s = model.addVars(jobs, [1, 2], name="s", vtype=GRB.CONTINUOUS, lb = 0.0)  # Start times
c = model.addVars(jobs, [1, 2], name="c", vtype=GRB.CONTINUOUS, lb = 0.0)  # Completion times
x = model.addVars(jobs, [1, 2], machines, name="x", vtype=GRB.BINARY)  # Machine assignment
T = model.addVars(jobs, name="T", vtype=GRB.CONTINUOUS, lb = 0.0)  # Tardiness
seq = model.addVars(jobs, jobs, [1, 2], vtype=GRB.BINARY, name="seq")  # Sequencing binary variables


M = 10000

# Constraints

# 1. Job assignment for each stage
for j in jobs:
    model.addConstr(quicksum(x[j, 1, k] for k in machines if k in data.at[data.index[data['Job ID'] == j][0], 'Stage-1 Machines']) == 1)
    model.addConstr(quicksum(x[j, 2, k] for k in machines if k in data.at[data.index[data['Job ID'] == j][0], 'Stage-2 Machines']) == 1)

# 2. Processing time constraint
for j in jobs:
    p1 = data.loc[data['Job ID'] == j, 'Stage-1 Processing Time'].values[0]
    p2 = data.loc[data['Job ID'] == j, 'Stage-2 Processing Time'].values[0]
    model.addConstr(c[j, 1] == s[j, 1] + p1)
    model.addConstr(c[j, 2] == s[j, 2] + p2)

# 3. Stage order constraint
for j in jobs:
    model.addConstr(s[j, 2] >= c[j, 1])

# 4. Machine non-overlap for each stage

for i in jobs:
    for j in jobs:
        if i != j:
            model.addConstr(s[i, 1] + data.loc[data['Job ID'] == i, f'Stage-1 Processing Time'].values[0] <= s[j, 1] + M * (1 - seq[i, j, 1]))
            model.addConstr(s[j, 2] + data.loc[data['Job ID'] == j, f'Stage-2 Processing Time'].values[0] <= s[i, 2] + M * seq[i, j, 2])
            model.addConstr(s[i, 1] + data.loc[data['Job ID'] == i, f'Stage-1 Processing Time'].values[0] <= s[j, 2] + M * (1 - seq[i, j, 2]))
            model.addConstr(s[j, 2] + data.loc[data['Job ID'] == j, f'Stage-2 Processing Time'].values[0] <= s[i, 1] + M * seq[i, j, 1])


# 5. Linearized tardiness calculation
for j in jobs:
    d = data.loc[data['Job ID'] == j, 'Due Time'].values[0]
    model.addConstr(T[j] >= c[j, 2] - d)
    model.addConstr(T[j] >= 0)

# Optimize the model
model.optimize()

if model.status == GRB.INFEASIBLE:
    print("Model is infeasible; computing IIS")
    model.computeIIS()
    model.write("model.ilp")

if model.status == GRB.OPTIMAL:
    print(f"Total Tardiness: {model.objVal}")
    for j in jobs:
        print(f"Job {j} Tardiness: {T[j].X}")
        for stage in [1, 2]:
            print(f"Stage {stage} start time: {s[j, stage].X}, completion time: {c[j, stage].X}")


TypeError: can only concatenate list (not "tuple") to list